In [1]:
from flask import Flask, request, render_template, redirect,url_for,make_response,send_from_directory, session, Markup
from gevent import pywsgi
import os, time, markdown, sqlite3

In [2]:
app = Flask(__name__)
app.config['SECRET_KEY'] = os.urandom(24)
app.config['FILE_PATH'] = 'static/passages/'

In [3]:
#md转html的方法 
def md2html(filename):
    exts = ['markdown.extensions.extra', 'markdown.extensions.codehilite','markdown.extensions.tables','markdown.extensions.toc']
    mdcontent = ""
    with open(filename,'r',encoding='utf-8') as f:
        mdcontent = f.read()
        pass
    html = markdown.markdown(mdcontent,extensions=exts)
    content = Markup(html)
    return content

In [4]:
database=sqlite3.connect('database')
cursor=database.cursor()
cursor.execute('create table if not exists ttou(title text,category text,url text,time text, cover text)')
database.commit()

In [5]:
@app.route('/', methods=['GET'])
def index():
    database=sqlite3.connect('database')
    cursor=database.cursor()
    cursor.execute('select * from ttou')
    records = cursor.fetchall()
    categories = []
    for i in records:
        if i[1] not in categories:
            categories.append(i[1])
    return render_template('index.html', category = categories, records=records)

In [6]:
@app.route('/<category>', methods=['GET'])
def category(category):
    database=sqlite3.connect('database')
    cursor=database.cursor()
    cursor.execute('select * from ttou')
    records = cursor.fetchall()
    passages = []
    categories = []
    for i in records:
        if i[1] == category:
            passages.append(i)
        if i[1] not in categories:
            categories.append(i[1])
    return render_template('category.html', category = categories, records=passages)

In [7]:
@app.route('/login', methods=['GET','POST'])
def login():
    if request.method == 'GET':
        return render_template('login.html')
    elif request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')
        session['username'] = username
        session['password'] = password
        if (session.get('username')=='lankning') and (session.get('password')=='123456'):
            return redirect(url_for("admin"))
        else:
            return redirect(url_for("login"))

In [8]:
@app.route('/admin', methods=['GET','POST'])
def admin():
    if request.method == 'GET':
        if (session.get('username')=='lankning') and (session.get('password')=='123456'):
            return render_template('admin.html')
        else:
            return redirect(url_for("login"))
    elif request.method == 'POST':
        page = request.form.get('button')
        # print(page)
        return redirect(url_for(page))

In [9]:
@app.route('/admin/modify', methods=['GET','POST'])
def modify():
    if request.method == 'GET':
        if (session.get('username')=='lankning') and (session.get('password')=='123456'):
            database=sqlite3.connect('database')
            cursor=database.cursor()
            cursor.execute('select * from ttou')
            records = cursor.fetchall()
            return render_template('modify.html',records=records)
        else:
            return redirect(url_for("login"))
    elif request.method == 'POST':
        title = request.form.get('delete')
        database=sqlite3.connect('database')
        cursor=database.cursor()
        cursor.execute('select * from ttou')
        records=cursor.fetchall()
        for i in records:
            if i[0]==title:
                category = i[1]
        os.remove(os.path.join(app.config['FILE_PATH'],category,title+'.md'))
        # print(title,category,'deleted')
        cursor.execute('DELETE FROM ttou WHERE title=\"%s\"' % title)
        database.commit()
        return redirect(url_for("modify"))

In [10]:
@app.route('/admin/new', methods=['GET','POST'])
def new():
    if request.method == 'GET':
        if (session.get('username')=='lankning') and (session.get('password')=='123456'):
            return render_template('new.html')
        else:
            return redirect(url_for("login"))
    elif request.method == 'POST':
        title = request.form.get('title')
        category = request.form.get('category')
        text = request.form.get('text')
        if request.form.get('time')=='Default':
            localtime = time.asctime( time.localtime(time.time()) )
        else:
            localtime =  request.form.get('time')
        cover =  request.form.get('cover')
        context = '# '+title+'\n'+text+'\n'+localtime
#         print(context)
        catepath = os.path.join(app.config['FILE_PATH'],category)
        if not os.path.exists(catepath):#判断存放图片的文件夹是否存在
            os.makedirs(catepath) # 若图片文件夹不存在就创建
        with open('%s.md'% (os.path.join(catepath,title)), "w+",encoding='utf-8') as m:
            m.write(context)
        target = category+'+'+title
        database=sqlite3.connect('database')
        cursor=database.cursor()
        cursor.execute('insert into ttou (title,category,url,time,cover) VALUES (\"%s\",\"%s\",\"%s\",\"%s\",\"%s\")' 
                       %(title,category,target,time,cover))
        database.commit()
        return redirect(url_for("read",target=target))

In [11]:
@app.route('/read/<target>', methods=['GET'])
def read(target):
#     print(target)
    database=sqlite3.connect('database')
    cursor=database.cursor()
    cursor.execute('select * from ttou')
    records = cursor.fetchall()
    categories = []
    for i in records:
        if i[1] not in categories:
            categories.append(i[1])
    target = target.replace('+','/')
    html = md2html('%s.md'% (os.path.join(app.config['FILE_PATH'],target)))
    return render_template('read.html',content = html, category = categories, records=records)

In [ ]:
if __name__ == '__main__':
    port=5000
    server = pywsgi.WSGIServer(('0.0.0.0', port), app)
    server.serve_forever()

127.0.0.1 - - [2020-07-08 13:36:57] "GET /read/%E5%AD%A6%E7%A7%91%E7%AC%94%E8%AE%B0+%E5%85%88%E8%BF%9B%E5%88%B6%E9%80%A0%E6%8A%80%E6%9C%AF%E8%AF%BE%E7%A8%8B%E7%AC%94%E8%AE%B0 HTTP/1.1" 200 13846 0.061837
127.0.0.1 - - [2020-07-08 13:36:57] "GET /static/default.css HTTP/1.1" 404 356 0.002991
127.0.0.1 - - [2020-07-08 13:36:57] "GET /static/github.css HTTP/1.1" 404 356 0.001993
127.0.0.1 - - [2020-07-08 13:37:10] "GET /static/default.css HTTP/1.1" 404 356 0.000999
127.0.0.1 - - [2020-07-08 13:37:10] "GET /static/github.css HTTP/1.1" 404 356 0.000997
